In [ ]:
%cd "C:\\Users\\Administrator\\Desktop\\强化学习\\螺纹钢\\10分钟\\"
path = "C:\\Users\\Administrator\\Desktop\\强化学习\\螺纹钢\\10分钟\\"

import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import random
from env.futures_portfoilo_env_deduct_immediately import *
from stable_baselines3 import *
from stable_baselines3.common.callbacks import *
from stable_baselines3.common.vec_env import *

import talib
import torch
import os
import optuna
from optuna.visualization import *
from tqdm import tqdm

# Read Data

In [ ]:
train_obs = np.load(path+'Processed_data\\train_obs.npy', allow_pickle=True).astype('float32')
train_symbols = np.load(path+'Processed_data\\train_symbols.npy', allow_pickle=True)
train_price = np.load(path+'Processed_data\\train_price.npy', allow_pickle=True)
valid_factors = np.load(path+'Processed_data\\'+'valid_factors.npy', allow_pickle=True).tolist()
stock_num = len(train_symbols[0])-1

train_obs.shape

# Data Quality Check

In [ ]:
zeros = []
names = []
columns = ['open', 'high', 'low', 'close', 'volume', 'amount', 'vwap', 'open_interest'] + valid_factors
for i in range(train_obs.shape[-1]):
    tmp_zeros = (train_obs[np.where(train_obs[:,:,i]==0)].shape[0])
    names.append(columns[i])
    zeros.append(tmp_zeros)
    
data_quality = pd.DataFrame()
names.append('Total Zeros')
zeros.append(train_obs[np.where(train_obs==0)].shape[0])
names.append('Total Zeros Pct')
zeros.append((train_obs[np.where(train_obs==0)].shape[0])/(train_obs.flatten().shape[0]))
data_quality['name'] = names
data_quality['zeros'] = zeros

data_quality

# Callback Function

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, log_dir, max_round, serial_num, early_stop=True, window=1, features=1, ema=1, std=1, \
                test_env=None, test_obs=None):
        super(SaveOnBestTrainingRewardCallback, self).__init__() 
        self.log_dir = log_dir
        self.best_reward = -np.inf
        self.best_sharpe = -np.inf
        self.best_return_1 = -np.inf
        self.best_sharpe_1 = -np.inf
        self.round = 0
        self.max_round = max_round
        self.early_stop = early_stop
        self.early_stop_counter = 0
        self.serial_num = serial_num
        self.last_reward = 0
        self.window = window
        self.features = features
        self.test_env = test_env
        self.test_obs = test_obs
        self.ema = ema
        self.std = std
    
    def _init_callback(self) -> None:
        # Create folder if needed
        if self.log_dir is not None:
            os.makedirs(self.log_dir, exist_ok=True)

    def _on_step(self) -> bool:
        done = self.locals['infos'][0]['done']
        if done == True:
            return_1 = self.locals['infos'][0]['return rate']
            maxdrawn_1, maxdrawn_duration_1  = self.locals['infos'][0]['maxdrawndown']
            sharpe_1 = self.locals['infos'][0]['sharpe']
            long_trades = self.locals['infos'][0]['long trades']
            short_trades = self.locals['infos'][0]['short trades']
            print('')

            if return_1 == 0 or return_1 == self.last_reward:
                self.early_stop_counter += 1
            else:
                pass
            if return_1 > self.best_return_1:
                self.best_return_1 = return_1
            else:
                pass
            if sharpe_1 > self.best_sharpe_1:
                self.best_sharpe_1 = sharpe_1
            else:
                pass       

            current_time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
            print('------------------Callback Start------------------')
            print("Round", self.round, ' has finished at ', current_time)
            print(" Current Best Return Rate: " ,self.best_return_1, ', Current Best Sharpe: ', self.best_sharpe_1)
            if return_1 >= 0.5 and sharpe_1 >= 1 and long_trades != 0 and short_trades != 0:
                self.model.save(self.log_dir+"best_model_"+self.serial_num+"_"+str(self.window)+'_'+str(self.features)\
                                +'_'+str(self.ema)+'_'+str(self.std)+'_'+str(return_1)+'_'+current_time)
                print("Saving new best model to: ", self.log_dir+"best_model_"+self.serial_num+"_"+str(self.window)\
                    +'_'+str(self.features)+'_'+str(self.ema)+'_'+str(self.std)+'_'+str(return_1)+'_'+current_time)                       
            else:
                pass
            print('------------------Callback Ends-------------------')
            print("")             
            self.round += 1
            self.last_reward = return_1
            
            try:
                os.remove(self.log_dir+self.serial_num+'.npy')
            except:
                pass
            np.save(self.log_dir+self.serial_num+'.npy', self.best_return_1)
            
        else:
            pass
        if self.round >= self.max_round:
            continue_training = False
            return continue_training
        elif self.early_stop == True and self.early_stop_counter > 2:
            continue_training = False
            return continue_training
        else:
            continue_training = True
            return continue_training

In [ ]:
log_dir = path+"best_models\\"
if os.path.exists(log_dir) == True:
    pass
else:
    os.makedirs(log_dir) 

# Parameter Optimization

In [ ]:
#Parameter Optimization
def objective(trial:optuna.Trial):
    torch.cuda.empty_cache()
    torch.cuda.set_device(0)
    alphabet = 'abcdefghijklmnopqrstuvwxyz1234567890'
    characters = str(random.sample(alphabet, 6)).replace(",","").replace("'","").replace("[","").replace("]","").replace(" ","")
    #################################################################################################################################
    num_env = 2#trial.suggest_int("num_env", 1, 3, 1)
    return_style = 1#trial.suggest_int("return_style", 1, 2, 1)
    std_period = 150
    window_size = int(trial.suggest_int("window_size", 30, train_obs.shape[1], 2))
    num_features = int(trial.suggest_int("num_features", 5, train_obs.shape[-1], 1))
    EMA_period = 150
    tmp_train_obs = train_obs[:,-window_size:,:num_features]
    tmp_train_price = train_price
    tmp_train_symbols = train_symbols
        
    print(tmp_train_obs.shape, tmp_train_price.shape, tmp_train_symbols.shape, EMA_period, std_period)
    #################################################################################################################################
    n_layers_value = trial.suggest_int("n_layers_value", 5, 100, 1)#100
    hidden_size_value = trial.suggest_int("hidden_size_value", 16, 1024, 8)#1024
    gamma = trial.suggest_loguniform("gamma", 0.01, 1)
    normalize_advantage = trial.suggest_categorical("normalize_advantage", [False, True])
    max_grad_norm = trial.suggest_loguniform("max_grad_norm", 0.01, 10)
    use_rms_prop = trial.suggest_categorical("use_rms_prop", [False, True])
    gae_lambda = trial.suggest_loguniform("gae_lambda", 0.1, 1.0)
    n_steps = trial.suggest_categorical("n_steps", [8, 16, 32, 64, 128, 256, 512, 1024, 2048])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
    ent_coef = trial.suggest_loguniform("ent_coef", 0.00000001, 0.1)
    vf_coef = trial.suggest_uniform("vf_coef", 0, 1)
    #################################################################################################################################
    slippage = 2
    commission_rate = 0.00011
    env = TradingEnv_2(obs=tmp_train_obs, price=tmp_train_price, symbols=tmp_train_symbols, initial_fund=15000, 
                       pos_factor=1, both_pos=True, short_only=False, long_only=False, num_stock=stock_num, 
                       commission_rate=commission_rate, valid_folder=None, mimimum_share=1, slippage=slippage, 
                       commission=None, test=False, valid=False, active_shares=False, cash_out_stop=False, 
                       returns=2, calculate_fund=False, action_type=return_style)
    rounds = 25
    save_best = SaveOnBestTrainingRewardCallback(log_dir, rounds, characters, True, window_size, num_features, 
                                                 EMA_period, std_period)
    rounds_of_train = (tmp_train_obs.shape[0])*rounds
    net_pi = []
    net_vf = []
    for i in range(n_layers_value):
        net_pi.append(hidden_size_value)
        net_vf.append(hidden_size_value)
    policy_kwargs = dict(net_arch=[dict(pi=net_pi, vf=net_vf)])
    print('Now Using GPU No.', torch.cuda.current_device())
    #################################################################################################################################
    try:
        model = A2C(policy='MlpPolicy', env=env, learning_rate=learning_rate, n_steps=n_steps, 
                    gamma=gamma, vf_coef=vf_coef, max_grad_norm=max_grad_norm, use_rms_prop=use_rms_prop, 
                    normalize_advantage=normalize_advantage, verbose=1, seed=123456, device='cuda', 
                    gae_lambda=gae_lambda, ent_coef=ent_coef, policy_kwargs=policy_kwargs)
        model.learn(total_timesteps=rounds_of_train, callback=save_best, log_interval=None)
        sharpe = np.load(log_dir+characters+'.npy', allow_pickle=True)
        os.remove(log_dir+characters+'.npy')
        torch.cuda.empty_cache()
    except Exception as e:
        print()
        print('********************Error Info Start********************')
        print(e)
        print('********************Error Info End********************')
        print()
        sharpe = -np.inf
        torch.cuda.empty_cache()
    model = None
    return sharpe
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, n_jobs=1)

In [ ]:
matplotlib.plot_optimization_history(study)

In [ ]:
matplotlib.plot_param_importances(study)

In [ ]:
matplotlib.plot_slice(study, params=["n_layers_value", "hidden_size_value"])